This notebook illustrates the code used in the paper ```U. Dobramysl, D. Holcman, Mixed analytical-stochastic simulation method for the recovery of a Brownian gradient source from probability fluxes to small windows, Journal of Computational Physics 355 (2018)```.

To run this notebook, you need a Python 3 installation (e.g. Anaconda) and the python packages `cython`, `numpy`, `scipy`, `matplotlib`, `seaborn`, `multiprocess` and `tqdm`. To install them, simply run:

```pip install cython numpy scipy matplotlib seaborn multiprocess tqdm```

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from utils import json_cache_write, json_cache_read, send_parallel_jobs
import pyximport; pyximport.install()
from receptor_fluxes import SimulationDomain
rc("figure", figsize=(8,6))
sns.set_context("poster")
sns.set_style("ticks"); None

# Simulation of particle fluxes to receptors at steady state - disk

In [ ]:
def run_simulation(parameters):
    """Perform a single realisation of the simulation.
    
    Args:
        source_distance (float): Distance to the source x.
        thetas (list|floats): Receptor angles.
        nr_particles (int): Number of particles to release.
    Returns:
        Receptor fluxes.
    """
    from receptor_fluxes import SimulationDomain

    params = {
        "outer_radius": 1.3,
        "nr_particles": 1e4,
    }
    params.update(parameters)
    
    if params['source_distance'] < 1.0:
        return {"parameters": params,
                "results": 
                {"normalized_receptor_fluxes": [float("NaN"), float("NaN")]}}
    
    sd = SimulationDomain(params["thetas"], source_distance=params["source_distance"],
                          outer_radius=params["outer_radius"])
    for _ in range(int(params["nr_particles"])):
        sd.release_particle()
    return {"parameters": params, "results": {"normalized_receptor_fluxes": sd.receptor_fluxes()[1].tolist()}}

## Simulation Results

In [ ]:
def plot_fluxes(name, fix=[], legend=None, axlabel="", labelpos=0.3):
    from collections import defaultdict
    from scipy.interpolate import interp1d
    data = defaultdict(lambda : defaultdict(list))
    for res in json_cache_read(name):
        params = res["parameters"]
        fluxes = res["results"]["normalized_receptor_fluxes"]
        data[params['source_distance']][params['thetas'][1]].append(fluxes[1])
    maxf = 0.0
    minf = 1.0
    styles = iter(['-', '--', '-.', ':'])
    if not isinstance(labelpos, list):
        labelpos = len(data)*[labelpos]
    for j, x in enumerate(sorted(data)):
        d = data[x]
        theta = sorted(d)
        fluxes = [np.mean(d[t]) for t in theta]
        maxf = max(max(fluxes), maxf)
        minf = min(min(fluxes), minf)
        for f in fix:
            fluxes[f] = float("NaN")
        l = plt.plot(theta, fluxes, color='black', ls=next(styles), label="$L=%g$" % x)[0]
    maxt = max(theta)
    plt.xlabel(r"Receptor 2 angle $\theta_2$")
    plt.ylabel(r"Splitting probability $J_2/(J_1+J_2)$")
    plt.xticks([0, np.pi/4, np.pi/2, np.pi*3/4., np.pi, np.pi*5/4., np.pi*3/2., np.pi*7/4., 2*np.pi],
               ["$0$", r"$\pi/4$", r"$\pi/2$", r"$3\pi/4$", r"$\pi$", r"$5\pi/4$", r"$3\pi/2$", r"$7\pi/8$", r"$2\pi$"])
    plt.axis(xmax=1*maxt, ymin=min(0.45, 0.95*minf), ymax=max(0.51, maxf))
    if legend is not None:
        plt.legend(loc=legend)
    plt.text(0.05, 0.05, axlabel, transform=plt.gca().transAxes)
    # sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    # sns.despine()
    plt.axis(ymin=0, ymax=1)

### Receptor 1 at $\theta_1=0$, vary $\theta_2$

In [ ]:
todo = sum([10*[{"source_distance": x, "thetas":[0, angle], 'nr_particles': 1e3} for angle in np.linspace(0, np.pi, 50)]
            for x in [1.3, 5.0, 10.0, 100.0]], [])
results = send_parallel_jobs("zero_pi_flux", run_simulation, todo)
plot_fluxes("zero_pi_flux")

### Receptor 1 at $\theta_1=\pi$, vary $\theta_2$

In [ ]:
todo = sum([10*[{"source_distance": x, "thetas":[np.pi, angle], 'nr_particles': 1e3} for angle in np.linspace(0, np.pi, 50)]
            for x in [1.3, 5.0, 10.0, 100.0]], [])
results = send_parallel_jobs("pi_flux", run_simulation, todo)
plt.title(r"Simulation data for $\theta_1=\pi$")
plot_fluxes("pi_flux")

### Receptor 1 at $\pi/2$, vary $\theta_2$

In [ ]:
todo = sum([10*[{"source_distance": x, "thetas":[np.pi/2., angle], 'nr_particles': 1e3} for angle in np.linspace(0, 2*np.pi, 50)]
            for x in [1.3, 5.0, 10.0, 100.0]], [])
results = send_parallel_jobs("pi_half_flux", run_simulation, todo)
plt.title(r"Simulation data for $\theta_1=\pi/2$")
plot_fluxes("pi_half_flux", fix=[])

### Contour plots 

In [ ]:
x = np.linspace(0, 20, 21)
xx, yy = np.meshgrid(x, x)

def rotate_thetas(x, y):
    thetas = np.array([0, np.pi])
    phi = np.arctan2(y, x)
    return (thetas - phi).tolist()

todo = [{"source_distance": np.linalg.norm([x, y]), "thetas":rotate_thetas(x, y), 'nr_particles': 1e3} for x, y in zip(xx.flat, yy.flat)]
results = send_parallel_jobs("contour_zero_pi", run_simulations, todo)

In [ ]:
def plot_contours(name, symmetry, dtheta=0.0, axeslabel="", calcz=None, smoothen=None,
                  labelpos=[(0, 5), (0, 10), (-10, 2.5)]):
    from matplotlib.mlab import griddata
    from scipy.optimize import leastsq
    from skimage.filters import gaussian
    m = []
    for res in json_cache_read(name):
        p = res["parameters"]
        radius = p["source_distance"]
        angle = -p["thetas"][0]-dtheta
        x = np.round(radius*np.cos(angle), decimals=14)
        y = np.round(radius*np.sin(angle), decimals=14)
        fluxes = res["results"]["normalized_receptor_fluxes"]
        if calcz is None:
            calcz = lambda fluxes: abs(np.diff(fluxes))[0]
        z = calcz(fluxes)
        if not np.isfinite(z):
            z = 0.0
        m.append((x, y, z))
        if symmetry == "inflection":
            m.append((x, -y, z))
            m.append((-x, y, z))
            m.append((-x, -y, z))
        elif symmetry == "ymirror":
            m.append((x, -y, z))
    theta1 = dtheta
    theta2 = dtheta - np.diff(p["thetas"])[0]
    m = np.array(sorted(m))
    ax = plt.gca()
    current_palette = sns.color_palette()
    
    if smoothen is not None:
        xi = np.linspace(m[:, 0].min(), m[:, 0].max(), 1000)
        yi = np.linspace(m[:, 1].min(), m[:, 1].max(), 1000)
        zi = griddata(m[:, 0], m[:, 1], m[:, 2], xi, yi, interp='linear')
        zs = gaussian(zi, sigma=smoothen)
        CS = ax.contour(xi, yi, zs, [0.01, 0.05, 0.1], colors=current_palette)
        # ax.contourf(xi, yi, zs, [0.01, 0.05, 0.1, 1.0], colors=current_palette)
    else:
        CS = ax.tricontour(m[:, 0], m[:, 1], m[:, 2], [0.01, 0.05, 0.1], colors=current_palette)
        # ax.tricontourf(m[:, 0], m[:, 1], m[:, 2], [0.01, 0.05, 0.1, 1.0], colors=current_palette)
    
    for c in CS.collections:
        paths = c.get_paths()
        v = sorted(paths, key=lambda p: -len(p.vertices))[0].vertices

        def resi(c):
            c = np.array([c[0], c[1]])
            Ri = ((v-c[np.newaxis, :])**2).sum(axis=1)**0.5
            return Ri - Ri.mean()

        def meanrad(c):
            c = np.array([c[0], c[1]])
            Ri = ((v-c[np.newaxis, :])**2).sum(axis=1)**0.5
            return Ri.mean()
        
        print meanrad(leastsq(resi, (1,-5))[0])
    
    # ax.clabel(CS, CS.levels, inline=True, inline_spacing=20,
    #          fmt={l: ("%d %%" % (l*100)) for l in CS.levels},
    #          manual=labelpos,
    #          fontsize=12)
    for l, lpos in zip(CS.levels, labelpos):
        ax.annotate("%d %%" % (l*100), lpos, fontsize=12, ha="center", va="center")
    
    ax.add_patch(plt.Circle((0,0), radius=1, zorder=20, edgecolor="k", facecolor="w", linewidth=1))
    for i, t in enumerate(p["thetas"]):
        t -= p["thetas"][0] + dtheta
        ax.add_patch(plt.Circle((np.cos(t), np.sin(t)), radius=0.2, color='k', zorder=21))
    ax.axis('square')
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")

Maximal separation between receptors yields the following detection contours (blue is the $1\%$ threshold, green the $5\%$ threshold and red the $10\%$ threshold. The contoured data is the flux difference between the receptors $|J_1-J_2|/(J_1+J_2)$.

In [ ]:
plt.title(r"$\theta_1=\pi/2$ and $\theta_2=-\pi/2$")
plot_contours("contour_zero_pi", "inflection", np.pi/2.)

In [ ]:
x = np.linspace(-20, 20, 41)
y = np.linspace(0, 20, 21)
xx, yy = np.meshgrid(x, y)

def rotate_thetas(x, y):
    thetas = np.array([-np.pi/4., np.pi/4.])
    phi = np.arctan2(y, x)
    return (thetas - phi).tolist()

todo = [{"source_distance": np.linalg.norm([x, y]), "thetas": rotate_thetas(x, y), 'nr_particles': 1e3} for x, y in zip(xx.flat, yy.flat)]
results = send_parallel_jobs("contour_pi_quarter", run_simulations, todo)

The following plot shows the detection contours for a receptor angle separation of $\pi/2$.

In [ ]:
plt.title(r"$\theta_1=\pi/4$ and $\theta_2=-\pi/4$")
plot_contours("contour_pi_quarter", "ymirror", np.pi/4.)